# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 12:00AM,239033,10,0085744192,ISDIN Corporation,Released
1,Jun 8 2022 12:00AM,239033,10,0085744232,ISDIN Corporation,Released
2,Jun 8 2022 12:00AM,239033,10,0085741075,ISDIN Corporation,Released
3,Jun 8 2022 12:00AM,239033,10,0085744253,ISDIN Corporation,Released
4,Jun 8 2022 12:00AM,239033,10,0085744254,ISDIN Corporation,Released
5,Jun 8 2022 12:00AM,239033,10,0085744288,ISDIN Corporation,Released
6,Jun 8 2022 12:00AM,239033,10,0085744289,ISDIN Corporation,Released
7,Jun 8 2022 12:00AM,239033,10,0085744290,ISDIN Corporation,Released
8,Jun 8 2022 12:00AM,239033,10,0085745219,ISDIN Corporation,Released
9,Jun 8 2022 12:00AM,239033,10,0085745221,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,239030,Released,1
41,239031,Released,1
42,239033,Released,50
43,239035,Released,7
44,239036,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239030,NaN,NaN,1.0
239031,NaN,NaN,1.0
239033,NaN,NaN,50.0
239035,NaN,NaN,7.0
239036,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238940,0.0,25.0,5.0
238942,0.0,6.0,11.0
238943,9.0,9.0,6.0
238954,0.0,1.0,0.0
238956,3.0,10.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238940,0,25,5
238942,0,6,11
238943,9,9,6
238954,0,1,0
238956,3,10,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238940,0,25,5
1,238942,0,6,11
2,238943,9,9,6
3,238954,0,1,0
4,238956,3,10,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238940,,25,5
1,238942,,6,11
2,238943,9,9,6
3,238954,,1,
4,238956,3,10,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation
50,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc."
51,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd"
58,Jun 7 2022 3:27PM,239031,10,ISDIN Corporation
59,Jun 7 2022 3:27PM,239030,10,ISDIN Corporation
60,Jun 7 2022 3:23PM,239029,10,Yusen – 3D Matrix
61,Jun 7 2022 3:17PM,239028,10,Yusen – 3D Matrix
62,Jun 7 2022 3:12PM,239027,10,Yusen – 3D Matrix
63,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc."
68,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,,,50
1,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc.",,,1
2,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd",,,7
3,Jun 7 2022 3:27PM,239031,10,ISDIN Corporation,,,1
4,Jun 7 2022 3:27PM,239030,10,ISDIN Corporation,,,1
5,Jun 7 2022 3:23PM,239029,10,Yusen – 3D Matrix,,,1
6,Jun 7 2022 3:17PM,239028,10,Yusen – 3D Matrix,,,1
7,Jun 7 2022 3:12PM,239027,10,Yusen – 3D Matrix,,,1
8,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc.",,4,1
9,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc.",,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50,,
1,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc.",1,,
2,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd",7,,
3,Jun 7 2022 3:27PM,239031,10,ISDIN Corporation,1,,
4,Jun 7 2022 3:27PM,239030,10,ISDIN Corporation,1,,
5,Jun 7 2022 3:23PM,239029,10,Yusen – 3D Matrix,1,,
6,Jun 7 2022 3:17PM,239028,10,Yusen – 3D Matrix,1,,
7,Jun 7 2022 3:12PM,239027,10,Yusen – 3D Matrix,1,,
8,Jun 7 2022 2:57PM,239025,10,"Methapharm, Inc.",1,4,
9,Jun 7 2022 2:55PM,239023,10,"Methapharm, Inc.",21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50,,
1,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc.",1,,
2,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd",7,,
3,Jun 7 2022 3:27PM,239031,10,ISDIN Corporation,1,,
4,Jun 7 2022 3:27PM,239030,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50.0,NaN,NaN
1,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd",7.0,NaN,NaN
3,Jun 7 2022 3:27PM,239031,10,ISDIN Corporation,1.0,NaN,NaN
4,Jun 7 2022 3:27PM,239030,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 12:00AM,239033,10,ISDIN Corporation,50.0,0.0,0.0
1,Jun 7 2022 4:03PM,239036,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 7 2022 4:00PM,239035,19,"AdvaGen Pharma, Ltd",7.0,0.0,0.0
3,Jun 7 2022 3:27PM,239031,10,ISDIN Corporation,1.0,0.0,0.0
4,Jun 7 2022 3:27PM,239030,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3585208,105.0,27.0,1.0
12,238984,1.0,1.0,0.0
15,477883,11.0,34.0,9.0
18,478026,2.0,0.0,0.0
19,717016,8.0,1.0,0.0
20,1911881,138.0,20.0,3.0
21,717041,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3585208,105.0,27.0,1.0
1,12,238984,1.0,1.0,0.0
2,15,477883,11.0,34.0,9.0
3,18,478026,2.0,0.0,0.0
4,19,717016,8.0,1.0,0.0
5,20,1911881,138.0,20.0,3.0
6,21,717041,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,105.0,27.0,1.0
1,12,1.0,1.0,0.0
2,15,11.0,34.0,9.0
3,18,2.0,0.0,0.0
4,19,8.0,1.0,0.0
5,20,138.0,20.0,3.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,12,Released,1.0
2,15,Released,11.0
3,18,Released,2.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21
Status,,,,,,,
Completed,1.0,0.0,9.0,0.0,0.0,3.0,0.0
Executing,27.0,1.0,34.0,0.0,1.0,20.0,0.0
Released,105.0,1.0,11.0,2.0,8.0,138.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21
0,Completed,1.0,0.0,9.0,0.0,0.0,3.0,0.0
1,Executing,27.0,1.0,34.0,0.0,1.0,20.0,0.0
2,Released,105.0,1.0,11.0,2.0,8.0,138.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21
0,Completed,1.0,0.0,9.0,0.0,0.0,3.0,0.0
1,Executing,27.0,1.0,34.0,0.0,1.0,20.0,0.0
2,Released,105.0,1.0,11.0,2.0,8.0,138.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()